In [13]:
import requests
import time
import json
from bs4 import BeautifulSoup


PTT_URL = 'https://www.ptt.cc'

def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}  # 告知Server已回答過滿18歲的問題
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html5lib')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').text.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)  # 轉換字串為數字
                except ValueError:
                    # 若轉換失敗，可能是'爆'或 'X1', 'X2', ...
                    # 若不是, 不做任何事，push_count 保持為 0
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10
            base_url = 'https://www.ptt.cc'
            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').text
                author = d.find('div', 'author').text if d.find('div', 'author') else ''
                articles.append({
                    'title': title,
                    'href': base_url + href,
                    'push_count': push_count,
                    'author': author
                })
    return articles, prev_url


def get_author_ids(posts, pattern):
    ids = set()
    for post in posts:
        if pattern in post['author']:
            ids.add(post['author'])
    return ids

def hot():
    current_page = get_web_page(PTT_URL + '/bbs/Gossiping/index.html')
    if current_page:
        articles = []  # 全部的今日文章
        today = time.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        
        current_articles, prev_url = get_articles(current_page, today)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, today)

        # 印出所有不同的 5566 id
        # print(get_author_ids(articles, '5566'))

        # 儲存或處理文章資訊
        print('今天有', len(articles), '篇文章')
        threshold = 50
        print('熱門文章(> %d 推):' % (threshold))
        for a in articles:
            if int(a['push_count']) > threshold:
                print(a)
        with open('gossiping.json', 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

In [6]:
time.strftime("%m/%d").lstrip('0')

'2/26'

In [16]:
hot()

今天有 867 篇文章
熱門文章(> 50 推):
{'title': '[臉書] 柯文哲', 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1551158486.A.2F8.html', 'push_count': 52, 'author': 'rickey1270'}
{'title': 'Re: [新聞] 拒矮化!以色列全球市長會議 柯文哲「台灣', 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1551155999.A.322.html', 'push_count': 78, 'author': 'ERT312'}
{'title': '[新聞] 中科院極機密運載火箭「麒麟專案」外洩！', 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1551156085.A.E65.html', 'push_count': 99, 'author': 'zeuga'}
{'title': 'Re: [臉書] 赤燭', 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1551156534.A.E71.html', 'push_count': 68, 'author': 'artpoet'}
{'title': '[新聞] 張安樂率眾抬棺到立院抗議 從載棺貨車上', 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1551155845.A.327.html', 'push_count': 73, 'author': 'helikon'}
{'title': '[新聞] 推出第七日 《還願》全球下架', 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1551155628.A.E53.html', 'push_count': 57, 'author': 'killerken'}
{'title': 'Re: [新聞] 拒矮化!以色列全球市長會議 柯文哲「台灣', 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1551154809.A.F88.html', 'push_count': 76, 'a